In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from scipy import special
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
df = pd.read_csv("https://docs.employment.gov.au/system/files/doc/other/salm_smoothed_sa2_datafiles_asgs_2016_-_december_quarter_2019.csv", header=1) 

In [2]:
#Column Cleaning
df = df[1:]
df = df.rename(columns={'Data Item': 'Labour Force Category', 
                            'Statistical Area Level 2 (SA2) (2016 ASGS)': 'SA2_Name', 
                            'SA2 Code (2016 ASGS)': 'SA2_Code'
                           })

#Category Cleaning
df["Labour Force Category"]= df["Labour Force Category"].replace("Smoothed unemployment (persons)", "Unemployed")
df["Labour Force Category"]= df["Labour Force Category"].replace("Smoothed labour force (persons)", "Labour Force") 
df["Labour Force Category"]= df["Labour Force Category"].replace("Smoothed unemployment rate (%)", "Unemployment Rate") 

#SA2_Code from Float to Integer
df['SA2_Code'] = pd.to_numeric(df["SA2_Code"], downcast='signed')

#Creating State Code
df['State_Code'] = df['SA2_Code'].astype(str).str[0]

#Adding State Name
state_list = [
    ['NSW', '1'],
    ['VIC', '2'],
    ['QLD', '3'],
    ['SA', '4'],
    ['WA', '5'],
    ['TAS', '6'],
    ['NT', '7'],
    ['ACT', '8']
]
state_list = pd.DataFrame(state_list, columns = ['State_Name', 'State_Code'])

#Merging State Name into df (i.e like an Index/Match)
df = pd.merge(df, state_list, on='State_Code')

#Cleaning up the data and then re-joining it all together (First Part)
df1 = df.iloc[:,:3]
df2 = df.iloc[:,-2:]
starting_df = pd.concat([df1,df2], axis=1)
starting_df

#Cleaning up the data and then re-joining it all together (Second Part)
df3 = df.iloc[:,3:-2]
df3 = df3.replace('-','0')
df3 = df3.replace(',','', regex=True)
ending_df = df3.astype(float)
salm = pd.concat([starting_df,ending_df], axis=1)

#Replacing column names with dates
date_string = salm.columns[5:].to_list()
changed_date = []
for d in date_string:
    d = datetime.datetime.strptime(d, '%b-%y').strftime("%m/%Y")
    changed_date.append(d)

datetime_columns = pd.DataFrame(changed_date, columns =['Date'])
salm.columns.values[5:] = datetime_columns['Date']

#extracting the first few classification columns to then concat to the percentages
titles = salm.iloc[:,:5]

#adding quarterly percentages
percent_columns = salm.iloc[:,5:]
percent_columns = percent_columns.pct_change(axis=1)
percent_columns = percent_columns.fillna(0)
salm_percent = pd.concat([titles, percent_columns], axis=1)

#Adding in ID Columns
salm.insert(0, 'ID', range(1, 1+ len(salm)))
salm_percent.insert(0, 'ID', range(1, 1+ len(salm_percent)))

#Melting from wide to long
salm = pd.melt(salm, id_vars=['ID','Labour Force Category','SA2_Name','SA2_Code','State_Code','State_Name'])
salm_percent = pd.melt(salm_percent, id_vars=['ID','Labour Force Category','SA2_Name','SA2_Code','State_Code','State_Name'])

#Changing Column Name
salm = salm.rename(columns={'variable': 'Quarter', 'value': 'Level'})
salm_percent = salm_percent.rename(columns={'variable': 'Quarter', 'value': 'Growth'})

#Adjusting to Datetime
salm['Quarter'] = pd.to_datetime(salm['Quarter'])
salm_percent['Quarter'] = pd.to_datetime(salm_percent['Quarter'])

#Joining up ASGS Geographies
asgs = pd.read_csv("/Users/PeteKlestov/Scripts/Property/Property Monitor/01 Geography/SA1_2016_AUST.csv")
target_asgs = asgs[['SA2_MAINCODE_2016',
                    'SA2_NAME_2016',
                    'SA3_CODE_2016',
                    'SA3_NAME_2016',
                    'SA4_CODE_2016',
                    'SA4_NAME_2016',
                    'GCCSA_CODE_2016',
                    'STATE_NAME_2016']]

#Dropping duplicates, merging and dropping columns
target_asgs = target_asgs.drop_duplicates()

salm = pd.merge(salm, target_asgs, left_on='SA2_Code', right_on='SA2_MAINCODE_2016', how='inner')
salm_percent = pd.merge(salm_percent, target_asgs, left_on='SA2_Code', right_on='SA2_MAINCODE_2016', how='inner')

salm = salm.drop(['SA2_Name', 'SA2_Code', 'State_Code','State_Name'], axis=1)
salm_percent = salm_percent.drop(['SA2_Name', 'SA2_Code', 'State_Code','State_Name'], axis=1)

#Moving columns around
salm = salm[['ID',
             'SA2_MAINCODE_2016',
             'SA2_NAME_2016',
             'SA3_CODE_2016',
             'SA3_NAME_2016',
             'SA4_CODE_2016',
             'SA4_NAME_2016',
             'GCCSA_CODE_2016',
             'Labour Force Category',
             'Quarter',
             'Level']]

salm_percent = salm_percent[['ID',
             'SA2_MAINCODE_2016',
             'SA2_NAME_2016',
             'SA3_CODE_2016',
             'SA3_NAME_2016',
             'SA4_CODE_2016',
             'SA4_NAME_2016',
             'GCCSA_CODE_2016',
             'Labour Force Category',
             'Quarter',
             'Growth']]

#sorting and resetting the index
salm = salm.sort_values(by=['ID','Labour Force Category']).reset_index(drop=True)
salm_percent = salm_percent.sort_values(by=['ID','Labour Force Category']).reset_index(drop=True)
salm_ts = salm.set_index('Quarter')

In [3]:
salm.head()

,ID,SA2_MAINCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,Labour Force Category,Quarter,Level
0,1,101021008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2010-12-01,132.0
1,1,101021008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2011-03-01,99.0
2,1,101021008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2011-06-01,88.0
3,1,101021008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2011-09-01,83.0
4,1,101021008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2011-12-01,91.0


In [4]:
#comparing it to what's happening within the wider SA4
salm_benchmark = salm.groupby(['Quarter','Labour Force Category','SA4_NAME_2016']).mean()
salm_benchmark = salm_benchmark.drop(['ID','SA2_MAINCODE_2016','SA3_CODE_2016','SA4_CODE_2016'], axis=1)
salm_benchmark = salm_benchmark.reset_index()
salm_benchmark = salm_benchmark.rename(columns={'Level':'SA4 Average'})
salm_benchmark.head()

,Quarter,Labour Force Category,SA4_NAME_2016,SA4 Average
0,2010-12-01,Labour Force,Adelaide - Central and Hills,6568.041667
1,2010-12-01,Labour Force,Adelaide - North,6333.933333
2,2010-12-01,Labour Force,Adelaide - South,6022.562500
3,2010-12-01,Labour Force,Adelaide - West,6667.176471
4,2010-12-01,Labour Force,Australian Capital Territory,1941.952830


In [5]:
#merging average back onto original df
salm_benched = pd.merge(salm, salm_benchmark, on=['Quarter','Labour Force Category', 'SA4_NAME_2016'])
salm_benched['Difference'] = salm_benched['Level'] - salm_benched['SA4 Average']
salm_benched.head()

#add in moving average
#salm_benchmark['Moving Average'] = salm_benchmark['Level'].rolling(window=4).mean()

,ID,SA2_MAINCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,Labour Force Category,Quarter,Level,SA4 Average,Difference
0,1,101021008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2010-12-01,132.0,199.36,-67.36
1,2,101021009,Queanbeyan,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2010-12-01,209.0,199.36,9.64
2,3,101021010,Queanbeyan - East,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2010-12-01,50.0,199.36,-149.36
3,4,101021011,Queanbeyan Region,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2010-12-01,78.0,199.36,-121.36
4,5,101021012,Queanbeyan West - Jerrabomberra,10102,Queanbeyan,101,Capital Region,1RNSW,Unemployed,2010-12-01,97.0,199.36,-102.36


In [6]:
#getting the combined df into a time-series order for each SA2

salm_benched = salm_benched.sort_values(by=['SA2_MAINCODE_2016','Labour Force Category','Quarter']).reset_index()
salm_benched = salm_benched.drop(['index'], axis = 1)
salm_benched.head()

,ID,SA2_MAINCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,Labour Force Category,Quarter,Level,SA4 Average,Difference
0,556,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Labour Force,2010-12-01,1773.0,4166.076923,-2393.076923
1,556,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Labour Force,2011-03-01,1814.0,4263.961538,-2449.961538
2,556,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Labour Force,2011-06-01,1838.0,4322.576923,-2484.576923
3,556,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Labour Force,2011-09-01,1858.0,4372.461538,-2514.461538
4,556,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Labour Force,2011-12-01,1876.0,4416.730769,-2540.730769


In [7]:
salm_benched[(salm_benched['SA2_NAME_2016'] == 'Nhulunbuy') 
             & (salm_benched['Labour Force Category'] == 'Unemployment Rate')]

,ID,SA2_MAINCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,Labour Force Category,Quarter,Level,SA4 Average,Difference
228808,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2010-12-01,0.1,4.287500,-4.187500
228809,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2011-03-01,0.2,3.712500,-3.512500
228810,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2011-06-01,0.2,4.441667,-4.241667
228811,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2011-09-01,0.1,5.300000,-5.200000
228812,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2011-12-01,0.1,6.600000,-6.500000
228813,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2012-03-01,0.1,7.616667,-7.516667
228814,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2012-06-01,0.2,8.175000,-7.975000
228815,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2012-09-01,0.2,8.008333,-7.808333
228816,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2012-12-01,0.2,7.529167,-7.329167
228817,6193,702041064,Nhulunbuy,70204,East Arnhem,702,Northern Territory - Outback,7RNTE,Unemployment Rate,2013-03-01,0.2,8.379167,-8.179167


In [8]:
def check_suburb(suburb,category,asgs):
    associated_asgs = salm[(salm['SA2_NAME_2016'] == suburb)].drop(['SA3_CODE_2016','SA4_CODE_2016','Labour Force Category','Quarter','ID', 'SA2_MAINCODE_2016', 'Level'], axis=1).drop_duplicates()
    
    #inject the asgs
    asgs_name = associated_asgs[asgs].to_list()
    asgs_name = asgs_name[0]
    asgs_df = salm[(salm['Labour Force Category'] == category) & (salm[asgs] == asgs_name)]

    #averaging selection geography
    asgs_average = asgs_df.groupby(['Quarter',asgs]).mean()

    #removing unecesssary columns and cleaning
    asgs_consolidated = asgs_average.iloc[:,-1]
    asgs_consolidated = pd.DataFrame(asgs_consolidated)
    asgs_consolidated = asgs_consolidated.reset_index()
    asgs_consolidated = asgs_consolidated.set_index('Quarter')
    asgs_col_drop = [0]
    asgs_consolidated = asgs_consolidated.drop(asgs_consolidated.columns[asgs_col_drop], axis = 1)
    asgs_consolidated = asgs_consolidated.rename(columns={'Level':asgs_name})

    #grabbing suburb name
    suburb_name = salm_ts[(salm_ts['Labour Force Category'] == category) & (salm_ts['SA2_NAME_2016'] == suburb)]
    final_suburb_name = suburb_name[['Level']]
    final_suburb_name = final_suburb_name.rename(columns={'Level':suburb})

    #joining suburb and target asgs
    output_df = pd.merge(final_suburb_name, asgs_consolidated, on='Quarter')
    output_df = output_df.reset_index()
    
    #plotting
    plot_suburb = go.Scatter(
                    x=output_df.Quarter,
                    y=output_df.iloc[:,1],
                    name=suburb)

    plot_asgs = go.Scatter(
                    x=output_df.Quarter,
                    y=output_df.iloc[:,2],
                    name=asgs_name)

    data = [plot_suburb,plot_asgs]

    graph = iplot(data)
    return graph

In [12]:
check_suburb('Emerald','Unemployment Rate','SA4_NAME_2016')